In [ ]:
!pip install transformers

In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.metrics import f1_score

# load the model
model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

# set the prefix and suffix
prefix = "generate questions: "
suffix = " </s>"

# create a list of sentences
sentences = ["GVA4_Java and GVA5_Java are from same hypothetical_ancestor_ 1.",
             "hypothetical_ancestor_1 and GVA6_Java are from same hypothetical_ancestor_ 2.",
             "hypothetical_ancestor_2 and GVAB_Borneo are from same hypothetical_ancestor_ 3.",
             "GVA2_Malay_Penninsula and GVA3_Malay_Penninsula are from same hypothetical_ancestor_ 4.",
             "GVA1_Malay_Penninsula and hypothetical_ancestor_4 are from same hypothetical_ancestor_ 5.",
             "hypothetical_ancestor_3 and hypothetical_ancestor_5 are from same hypothetical_ancestor_ 6."]

# create empty lists for questions and references
questions = []
references = []

# generate questions
for sentence in sentences:
    input_text = prefix + sentence + suffix
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    outputs = model.generate(input_ids)
    question = tokenizer.decode(outputs[0], skip_special_tokens=True)
    questions.append(question)
    
    # add reference for calculating F1 score
    reference = sentence.split(" and ")[0].replace(".", "")
    references.append(reference)
    
# print questions and references
print("Generated Questions:")
print(questions)
print("\nReferences:")
print(references)

# calculate F1 score
f1 = f1_score(references, questions, average='macro')
print("\nF1 Score:", f1)


# **T5_BASE F1_Score**

In [ ]:
import torch
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.metrics import f1_score

# set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the pretrained model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# define the function to generate answers
def generate_answer(question, context):
    input_str = "question: " + question + " context: " + context #+ " </s>"
    input_ids = tokenizer.encode(input_str, return_tensors="pt").to(device)
    outputs = model.generate(input_ids)
    # print("Output")
    # print(tokenizer.decode(outputs[0]))
    tem = tokenizer.decode(outputs[0])
    ans = ""
    n = len(tem)
    f = True
    for c in range(0,n):
      if tem[c]=='<':
        f = False
      elif tem[c]=='>':
        f = True
      else:
        if f==True:
          ans = ans + tem[c]
    print(ans)
    return ans
    # return tokenizer.decode(outputs[0])

# define the function to calculate F1 score
def calculate_f1_score(true_labels, predicted_labels):
    return f1_score(true_labels, predicted_labels, average='weighted')

# example sentences
sentences = [
    "GVA4_Java and GVA5_Java are from same hypothetical_ancestor_ 1 .",
    "hypothetical_ancestor_1 and GVA6_Java are from same hypothetical_ancestor_ 2 .",
    "hypothetical_ancestor_2 and GVAB_Borneo are from same hypothetical_ancestor_ 3 .",
    "GVA2_Malay_Penninsula and GVA3_Malay_Penninsula are from same hypothetical_ancestor_ 4 .",
    "GVA1_Malay_Penninsula and hypothetical_ancestor_4 are from same hypothetical_ancestor_ 5 .",
    "hypothetical_ancestor_3 and hypothetical_ancestor_5 are from same hypothetical_ancestor_ 6 ."
]


questions = [
    ("Who are the parent of GVA4_Java?", "hypothetical_ancestor_ 1"),
    ("Who is the parent of GVA5_Java?", "hypothetical_ancestor_ 1"),
    ("Who is the parent of GVA4_Java and GVA5_Java?", "hypothetical_ancestor_ 1"),
    ("Who is the parent of GVA6_Java?", "hypothetical_ancestor_ 2"),
    ("Who is the parent of GVAB_Borneo?", "hypothetical_ancestor_ 3"),
    ("Who is the parent of GVA2_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("Who is the parent of GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("Who is the parent of GVA1_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("Who is the parent of GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),

    ("Who are the children of hypothetical_ancestor_2?", "hypothetical_ancestor_1 and GVA6_Java"),
    ("Who are the children of hypothetical_ancestor_4?", "GVA2_Malay_Penninsula and GVA3_Malay_Penninsula"),
    ("Who are the children of hypothetical_ancestor_1?", "GVA4_Java and GVA5_Java"),
    ("Who are the children of hypothetical_ancestor_3?", "hypothetical_ancestor_2 and GVAB_Borneo"),
    ("Who are the children of hypothetical_ancestor_5?", "GVA1_Malay_Penninsula and hypothetical_ancestor_4"),

    ("What is the common ancestor of GVA1_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA1_Malay_Penninsula,  GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("What is the common ancestor of GVA2_Malay_Penninsula and GVA1_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA4_Java and GVA5_Java?", "hypothetical_ancestor_ 1"),
    ("What is the common ancestor of GVA4_Java and GVA6_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVA6_Java and GVA5_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVA4_Java, GVA5_Java and GVA6_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVAB_Borneo and GVA6_Java?", "hypothetical_ancestor_ 3"),
    ("What is the common ancestor of GVAB_Borneo, GVA4_java, GVA5_Java and GVA6_Java?", "hypothetical_ancestor_ 3"),
    ("What is the common ancestor of GVAB_Borneo, GVA4_java, GVA5_Java?", "hypothetical_ancestor_ 3")
]

# generate answers and calculate F1 score
true_labels = []
predicted_labels = []
for question in questions:
    true_labels.append(question[1])
    answer = generate_answer(question[0], ' '.join(sentences))
    print(f"Q: {question[0]}")
    print(f"A: {answer}")
    predicted_labels.append(answer.strip())

f1 = calculate_f1_score(true_labels, predicted_labels)
# print("True Label")
# print(true_labels)
# print("Predicted Label")
# print(predicted_labels)
print(f"F1 score: {f1}")

In [ ]:
import torch
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.metrics import f1_score, precision_score, recall_score

# set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the pretrained model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-base")

# define the function to generate answers
def generate_answer(question, context):
    input_str = "question: " + question + " context: " + context #+ " </s>"
    input_ids = tokenizer.encode(input_str, return_tensors="pt").to(device)
    outputs = model.generate(input_ids)
    # print("Output")
    # print(tokenizer.decode(outputs[0]))
    tem = tokenizer.decode(outputs[0])
    ans = ""
    n = len(tem)
    f = True
    for c in range(0,n):
      if tem[c]=='<':
        f = False
      elif tem[c]=='>':
        f = True
      else:
        if f==True:
          ans = ans + tem[c]
    print(ans)
    return ans
    # return tokenizer.decode(outputs[0])

# define the function to calculate F1 score
def calculate_f1_score(true_labels, predicted_labels):
    return f1_score(true_labels, predicted_labels, average='weighted')

# example sentences
sentences = [
    "GVA4_Java and GVA5_Java are from same hypothetical_ancestor_ 1 .",
    "hypothetical_ancestor_1 and GVA6_Java are from same hypothetical_ancestor_ 2 .",
    "hypothetical_ancestor_2 and GVAB_Borneo are from same hypothetical_ancestor_ 3 .",
    "GVA2_Malay_Penninsula and GVA3_Malay_Penninsula are from same hypothetical_ancestor_ 4 .",
    "GVA1_Malay_Penninsula and hypothetical_ancestor_4 are from same hypothetical_ancestor_ 5 .",
    "hypothetical_ancestor_3 and hypothetical_ancestor_5 are from same hypothetical_ancestor_ 6 ."
]

questions = [
    ("Who are the parent of GVA4_Java?", "hypothetical_ancestor_ 1"),
    ("Who is the parent of GVA5_Java?", "hypothetical_ancestor_ 1"),
    ("Who is the parent of GVA4_Java and GVA5_Java?", "hypothetical_ancestor_ 1"),
    ("Who is the parent of GVA6_Java?", "hypothetical_ancestor_2"),
    ("Who is the parent of GVAB_Borneo?", "hypothetical_ancestor_ 3"),
    ("Who is the parent of GVA2_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("Who is the parent of GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("Who is the parent of GVA1_Malay_Penninsula?", "hypothetical_ancestor_5"),
    ("Who is the parent of GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),

    ("Who are the children of hypothetical_ancestor_2?", "hypothetical_ancestor_1 and GVA6_Java"),
    ("Who are the children of hypothetical_ancestor_4?", "GVA2_Malay_Penninsula and GVA3_Malay_Penninsula"),
    ("Who are the children of hypothetical_ancestor_1?", "GVA4_Java and GVA5_Java"),
    ("Who are the children of hypothetical_ancestor_3?", "hypothetical_ancestor_2 and GVAB_Borneo"),
    ("Who are the children of hypothetical_ancestor_5?", "GVA1_Malay_Penninsula and hypothetical_ancestor_4"),

    ("What is the common ancestor of GVA1_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA1_Malay_Penninsula,  GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("What is the common ancestor of GVA2_Malay_Penninsula and GVA1_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA4_Java and GVA5_Java?", "hypothetical_ancestor_ 1"),
    ("What is the common ancestor of GVA4_Java and GVA6_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVA6_Java and GVA5_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVA4_Java, GVA5_Java and GVA6_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVAB_Borneo and GVA6_Java?", "hypothetical_ancestor_ 3"),
    ("What is the common ancestor of GVAB_Borneo, GVA4_java, GVA5_Java and GVA6_Java?", "hypothetical_ancestor_ 3"),
    ("What is the common ancestor of GVAB_Borneo, GVA4_java, GVA5_Java?", "hypothetical_ancestor_ 3")
]

# generate answers and calculate F1 score
true_labels = []
predicted_labels = []
for question in questions:
    true_labels.append(question[1])
    answer = generate_answer(question[0], ' '.join(sentences))
    print(f"Q: {question[0]}")
    print(f"A: {answer}")
    predicted_labels.append(answer.strip())


y_true = true_labels
y_pred = predicted_labels
f1 = f1_score(y_true, y_pred, average="weighted")
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")

exact_match = 0
n = len(y_true)
for i in range(n):
  if y_true[i]==y_pred[i]:
    exact_match = exact_match + 1
#exact_match = int(y_true == y_pred)

# Print the output and metrics
print("Evaluation Metrics:")
print("F1-score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Exact Match:", exact_match)

# **T5_BioQA F1_Score**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained("ozcangundes/T5-base-for-BioQA").to(device)
tokenizer = T5Tokenizer.from_pretrained("ozcangundes/T5-base-for-BioQA")

# set up the device

# load the pretrained model and tokenizer
# model = T5ForConditionalGeneration.from_pretrained("t5-base").to(device)
# tokenizer = T5Tokenizer.from_pretrained("t5-base")


# define the function to generate answers
def generate_answer(question, context):
    input_str = "question: " + question + " context: " + context #+ " </s>"
    input_ids = tokenizer.encode(input_str, return_tensors="pt").to(device)
    outputs = model.generate(input_ids)
    # print("Output")
    # print(tokenizer.decode(outputs[0]))
    tem = tokenizer.decode(outputs[0])
    ans = ""
    n = len(tem)
    f = True
    for c in range(0,n):
      if tem[c]=='<':
        f = False
      elif tem[c]=='>':
        f = True
      else:
        if f==True:
          ans = ans + tem[c]
    print(ans)
    return ans
    # return tokenizer.decode(outputs[0])

# define the function to calculate F1 score
def calculate_f1_score(true_labels, predicted_labels):
    return f1_score(true_labels, predicted_labels, average='weighted')

# example sentences
sentences = [
    "GVA4_Java and GVA5_Java are from same hypothetical_ancestor_ 1 .",
    "hypothetical_ancestor_1 and GVA6_Java are from same hypothetical_ancestor_ 2 .",
    "hypothetical_ancestor_2 and GVAB_Borneo are from same hypothetical_ancestor_ 3 .",
    "GVA2_Malay_Penninsula and GVA3_Malay_Penninsula are from same hypothetical_ancestor_ 4 .",
    "GVA1_Malay_Penninsula and hypothetical_ancestor_4 are from same hypothetical_ancestor_ 5 .",
    "hypothetical_ancestor_3 and hypothetical_ancestor_5 are from same hypothetical_ancestor_ 6 ."
]


questions = [
    ("Who are the parent of GVA4_Java?", "hypothetical_ancestor_ 1"),
    ("Who is the parent of GVA5_Java?", "hypothetical_ancestor_ 1"),
    ("Who is the parent of GVA4_Java and GVA5_Java?", "same hypothetical_ancestor_ 1"),
    ("Who is the parent of GVA6_Java?", "hypothetical_ancestor_ 2"),
    ("Who is the parent of GVAB_Borneo?", "hypothetical_ancestor_3"),
    ("Who is the parent of GVA2_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("Who is the parent of GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("Who is the parent of GVA1_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("Who is the parent of GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "same hypothetical_ancestor_ 4"),

    ("Who are the children of hypothetical_ancestor_2?", "hypothetical_ancestor_1 and GVA6_Java"),
    ("Who are the children of hypothetical_ancestor_4?", "GVA2_Malay_Penninsula and GVA3_Malay_Penninsula"),
    ("Who are the children of hypothetical_ancestor_1?", "GVA4_Java and GVA5_Java"),
    ("Who are the children of hypothetical_ancestor_3?", "GVAB_Borneo"),
    ("Who are the children of hypothetical_ancestor_5?", "GVA1_Malay_Penninsula and hypothetical_ancestor_4"),

    ("What is the common ancestor of GVA1_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA1_Malay_Penninsula,  GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "same hypothetical_ancestor_ 4"),
    ("What is the common ancestor of GVA2_Malay_Penninsula and GVA1_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA4_Java and GVA5_Java?", "hypothetical_ancestor_ 1"),
    ("What is the common ancestor of GVA4_Java and GVA6_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVA6_Java and GVA5_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVA4_Java, GVA5_Java and GVA6_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVAB_Borneo and GVA6_Java?", "hypothetical_ancestor_ 3"),
    ("What is the common ancestor of GVAB_Borneo, GVA4_java, GVA5_Java and GVA6_Java?", "hypothetical_ancestor_ 3"),
    ("What is the common ancestor of GVAB_Borneo, GVA4_java, GVA5_Java?", "hypothetical_ancestor_ 3")
]

# generate answers and calculate F1 score
true_labels = []
predicted_labels = []
for question in questions:
    true_labels.append(question[1])
    answer = generate_answer(question[0], ' '.join(sentences))
    print(f"Q: {question[0]}")
    print(f"A: {answer}")
    predicted_labels.append(answer.strip())


y_true = true_labels
y_pred = predicted_labels
f1 = f1_score(y_true, y_pred, average="weighted")
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
#exact_match = int(y_true == y_pred)
exact_match = 0
n = len(y_true)
for i in range(n):
  if y_true[i]==y_pred[i]:
    exact_match = exact_match + 1

# Print the output and metrics
print("Evaluation Metrics:")
print("F1-score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Exact Match:", exact_match)

# **ALBERT F1_Score**

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ALBERT model and tokenizer
model_name = "mfeb/albert-xxlarge-v2-squad2"
model = AutoModelForQuestionAnswering.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# define the function to generate answers
def generate_answer(question, context):
    nlp = pipeline('question-answering', model=model, tokenizer=tokenizer, device=device)
    result = nlp(question=question, context=context)
    answer = result['answer']
    return answer

# define the function to calculate F1 score
def calculate_f1_score(true_labels, predicted_labels):
    return f1_score(true_labels, predicted_labels, average='weighted')

# example sentences
sentences = [
    "GVA4_Java and GVA5_Java are from same hypothetical_ancestor_ 1 .",
    "hypothetical_ancestor_1 and GVA6_Java are from same hypothetical_ancestor_ 2 .",
    "hypothetical_ancestor_2 and GVAB_Borneo are from same hypothetical_ancestor_ 3 .",
    "GVA2_Malay_Penninsula and GVA3_Malay_Penninsula are from same hypothetical_ancestor_ 4 .",
    "GVA1_Malay_Penninsula and hypothetical_ancestor_4 are from same hypothetical_ancestor_ 5 .",
    "hypothetical_ancestor_3 and hypothetical_ancestor_5 are from same hypothetical_ancestor_ 6 ."
]

questions = [
    ("Who are the parent of GVA4_Java?", "hypothetical_ancestor_1"),
    ("Who is the parent of GVA5_Java?", "hypothetical_ancestor_1"),
    ("Who is the parent of GVA4_Java and GVA5_Java?", "hypothetical_ancestor_1"),
    ("Who is the parent of GVA6_Java?", "hypothetical_ancestor_2"),
    ("Who is the parent of GVAB_Borneo?", "hypothetical_ancestor_ 3"),
    ("Who is the parent of GVA2_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("Who is the parent of GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("Who is the parent of GVA1_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("Who is the parent of GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),

    ("Who are the children of hypothetical_ancestor_2?", "hypothetical_ancestor_1 and GVA6_Java"),
    ("Who are the children of hypothetical_ancestor_4?", "GVA2_Malay_Penninsula and GVA3_Malay_Penninsula"),
    ("Who are the children of hypothetical_ancestor_1?", "GVA4_Java and GVA5_Java"),
    ("Who are the children of hypothetical_ancestor_3?", "hypothetical_ancestor_2 and GVAB_Borneo"),
    ("Who are the children of hypothetical_ancestor_5?", "GVA1_Malay_Penninsula and hypothetical_ancestor_4"),

    ("What is the common ancestor of GVA1_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA1_Malay_Penninsula,  GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA2_Malay_Penninsula and GVA3_Malay_Penninsula?", "hypothetical_ancestor_ 4"),
    ("What is the common ancestor of GVA2_Malay_Penninsula and GVA1_Malay_Penninsula?", "hypothetical_ancestor_ 5"),
    ("What is the common ancestor of GVA4_Java and GVA5_Java?", "hypothetical_ancestor_ 1"),
    ("What is the common ancestor of GVA4_Java and GVA6_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVA6_Java and GVA5_Java?", "hypothetical_ancestor_ 2"),
    ("What is the common ancestor of GVA4_Java, GVA5_Java and GVA6_Java?", "hypothetical_ancestor_ "),
    ("What is the common ancestor of GVAB_Borneo and GVA6_Java?", "hypothetical_ancestor_"),
    ("What is the common ancestor of GVAB_Borneo, GVA4_java, GVA5_Java and GVA6_Java?", "hypothetical_ancestor_ "),
    ("What is the common ancestor of GVAB_Borneo, GVA4_java, GVA5_Java?", "hypothetical_ancestor_ 3")
]

# generate answers and calculate F1 score
true_labels = []
predicted_labels = []
for question in questions:
    true_labels.append(question[1])
    answer = generate_answer(question[0], ' '.join(sentences))
    print(f"Q: {question[0]}")
    print(f"A: {answer}")
    predicted_labels.append(answer.strip())

y_true = true_labels
y_pred = predicted_labels
f1 = f1_score(y_true, y_pred, average="weighted")
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
#exact_match = int(y_true == y_pred)
exact_match = 0
n = len(y_true)
for i in range(n):
  if y_true[i]==y_pred[i]:
    exact_match = exact_match + 1

# Print the output and metrics
print("Evaluation Metrics:")
print("F1-score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Exact Match:", exact_match)
